### **Get Gas Data**

Sample notebook for loading gas data from IllustrisTNG. Loads data for the 10 most massive halos in TNG_300_3 at z=0 up to a radius of 2 Mpc/h.

In [7]:
import numpy as np
import illustris_python as il
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u

from IPython.core.display import display, HTML

%config IPCompleter.greedy = True
%config InlineBackend.figure_format ='retina'


def progress_bar(cur_val, final_val):
    """ 
    Function to keep track of progress during computations by displaying
    a progress bar

    Parameters:
    cur_val (int/float): current iteration/value calculation is on
    final_val (int/float): final iteration/value that calculation will take
    """

    bar_length = 20
    percent = float(cur_val) / final_val
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rProgress: [{0}]"
                     " {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()
    
basePath = '/global/cscratch1/sd/samgolds/IllustrisTNG/TNG-300_3/outputs/'
snap_dir_g = basePath+'snapdir_099/snap_099.0.hdf5'
snapshot_ind = 99

# Load header file and halo group catalog
header = il.groupcat.loadHeader(basePath, snapshot_ind)

halo_grp_fields = ['GroupCM', 'GroupMass', 'Group_M_Crit200', 'Group_M_Mean200', 
                   'Group_R_Crit200', 'Group_R_Mean200', 'GroupFirstSub']
halo_grp = il.groupcat.loadHalos(basePath, snapshot_ind, fields=halo_grp_fields)
halo_grp['has_gas'] = True


# Obtain basic cosmological info and obtain conversion factors
redshift = header['Redshift']
H = cosmo.H(redshift)
H0 = cosmo.H(0)
h = H0.value/100

# Define box boundary vector in Mpc (for period bcs)
boxsize = header['BoxSize'] # kpc/h
box_bounds = boxsize*np.ones(3)/1000

R_HALO = 2 # Size of sphere to select particles around halo_cm
n_halo = 9 # Select only the 10 most massive halos

halo_masses = halo_grp['GroupMass'][0:n_halo]*10**10/h

In [8]:
# Load all data of interest
fields = ['Coordinates', 'Velocities', 'Masses', 'InternalEnergy', 'ElectronAbundance', 'Density']
gas_data_chunk = il.snapshot.loadSubset(basePath, snapshot_ind, "gas", fields, sq=True)

# Initialize dictionary to store final information
final_data = np.array([dict() for x in range(n_halo)])

for i in range(n_halo):
    
    progress_bar(i, n_halo)
    # Get halo center of mass (Mpc/h)
    halo_cm = halo_grp['GroupCM'][i]/1000

    gas_coords_chunk = gas_data_chunk['Coordinates']/1000 # Gas coordinates in Mpc/h
    
    # Compute radius from cm
    dev = gas_coords_chunk-halo_cm

    # Account for halos which go over the boundary
    for ind, q in enumerate(dev.T):
        q = np.where(np.abs(q) > 0.5 * box_bounds[ind], box_bounds[ind]-np.abs(q), q)
        dev.T[ind] = q

    # Keep only particles within r = R_HALO Mpc/h
    r = np.linalg.norm(dev, axis=1)  
    
    filtered_indices = np.where(r < R_HALO)[0]
    
    for key in fields:
        final_data[i][key] = gas_data_chunk[key][filtered_indices]
        final_data[i]['GroupMass'] = halo_grp['GroupMass'][i]
        final_data[i]['GroupCM'] = halo_grp['GroupCM'][i]
        
    progress_bar(i+1, n_halo)
    
# Save results
np.save("Saved_Data/massive_halos_full_set.npy", final_data)